In [1]:
# Useful additional packages 
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
from math import pi


from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit import BasicAer, execute
from qiskit.quantum_info import Pauli, state_fidelity, basis_state, process_fidelity 

from qiskit.circuit import Parameter


from qiskit import *
IBMQ.load_accounts(hub=None)
from qiskit.tools.monitor import job_monitor, backend_monitor, backend_overview
from qiskit.providers.ibmq import least_busy

shots = 8192 
max_credits = 3 
#job_exp = execute(qc, backend=backend, shots=shots, max_credits=max_credits)
from qiskit.compiler import transpile

In [2]:
#Run in real quantum computer
#Misura di tutti i qubit
n = 4
q = QuantumRegister(n, 'q')
c = ClassicalRegister(n, 'c') #n

circ = QuantumCircuit(q,c)
phi = Parameter('phi')

circ.h(q[0])

for i in range(n-1):
    circ.cx(i, i+1)
    
circ.barrier()
circ.rz(phi, range(n))
circ.barrier()

for i in reversed(range(n-1)):
    circ.cx(i, i+1)

circ.h(q[0])

circ.barrier()

for i in range(n):
    circ.measure(q[i], c[i]) 

#circ.draw(output='mpl')

print('Phi range')
phi_range = [np.pi*j/(n+1) for j in range(2*n+1+1)]
print(phi_range)
circuits = [circ.bind_parameters({phi: phi_val}) for phi_val in phi_range] 
#print(circuits[3].parameters)
#circuits[11].draw(output='mpl')


job = execute(circ,
            backend=IBMQ.get_backend('ibmq_16_melbourne'),
            parameter_binds=[{phi: phi_val}
              for phi_val in phi_range],shots=shots)

job_monitor(job)

phi_range2 = phi_range
counts2 =[job.result().get_counts(i) for i in range(len(job.result().results))] 

print('\n')
print('Counts')
print(counts2)
print('\n')
print('Counts of 0000')
print(list(map(lambda c: 1/shots*c.get('0000', 0), counts2)))
job_id = job.job_id()
print('JOB ID: {}'.format(job_id))

Phi range
[0.0, 0.6283185307179586, 1.2566370614359172, 1.8849555921538759, 2.5132741228718345, 3.141592653589793, 3.7699111843077517, 4.39822971502571, 5.026548245743669, 5.654866776461628]
Job Status: job has successfully run


Counts
[{'1110': 67, '0000': 2068, '0111': 199, '1000': 296, '0101': 404, '0011': 186, '1010': 70, '1111': 29, '1101': 91, '1001': 235, '0001': 540, '1100': 323, '0100': 2813, '0110': 458, '1011': 47, '0010': 366}, {'1110': 39, '0000': 3535, '0111': 142, '1000': 456, '0101': 370, '0011': 285, '1010': 76, '1111': 22, '1101': 71, '1001': 140, '0001': 577, '1100': 245, '0100': 1488, '0110': 235, '1011': 34, '0010': 477}, {'1110': 23, '0000': 4411, '0111': 74, '1000': 437, '0101': 311, '0011': 380, '1010': 66, '1111': 21, '1101': 128, '1001': 176, '0001': 1146, '1100': 105, '0100': 250, '0110': 84, '1011': 53, '0010': 527}, {'1110': 22, '0000': 2422, '0111': 144, '1000': 309, '0101': 926, '0011': 438, '1010': 64, '1111': 33, '1101': 178, '1001': 306, '0001': 2415,

In [3]:
#Run in real quantum computer - Transpile
#Misura di tutti i qubit
n = 4
q = QuantumRegister(n, 'q')
c = ClassicalRegister(n, 'c') #n

circ = QuantumCircuit(q,c)
phi = Parameter('phi')

circ.h(q[0])

for i in range(n-1):
    circ.cx(i, i+1)
    
circ.barrier()
circ.rz(phi, range(n))
circ.barrier()

for i in reversed(range(n-1)):
    circ.cx(i, i+1)

circ.h(q[0])

circ.barrier()

for i in range(n):
    circ.measure(q[i], c[i]) 

#circ.draw(output='mpl')

print('Phi range')
phi_range = [np.pi*j/(n+1) for j in range(2*n+1+1)]
print(phi_range)
circuits = [circ.bind_parameters({phi: phi_val}) for phi_val in phi_range] 
#print(circuits[3].parameters)
#circuits[11].draw(output='mpl')

backend = IBMQ.get_backend('ibmq_16_melbourne')
trans_ghz = transpile(circ, backend)

job = execute(circ,
            backend,
            parameter_binds=[{phi: phi_val}
              for phi_val in phi_range],shots=shots)

job_monitor(job)

phi_range2 = phi_range
counts2 =[job.result().get_counts(i) for i in range(len(job.result().results))] 

print('\n')
print('Counts')
print(counts2)
print('\n')
print('Counts of 0000')
print(list(map(lambda c: 1/shots*c.get('0000', 0), counts2)))

job_id = job.job_id()
print('JOB ID: {}'.format(job_id))

#retrieved_job = backend.retrieve_job(job_id)
#retrieved_job.result().get_counts(qc)

Phi range
[0.0, 0.6283185307179586, 1.2566370614359172, 1.8849555921538759, 2.5132741228718345, 3.141592653589793, 3.7699111843077517, 4.39822971502571, 5.026548245743669, 5.654866776461628]
Job Status: job has successfully run


Counts
[{'1110': 42, '0000': 2058, '0111': 187, '1000': 282, '0101': 514, '0011': 149, '1010': 34, '1111': 23, '1101': 89, '1001': 253, '0001': 623, '1100': 375, '0100': 2925, '0110': 353, '1011': 29, '0010': 256}, {'1110': 26, '0000': 3644, '0111': 125, '1000': 479, '0101': 487, '0011': 213, '1010': 40, '1111': 24, '1101': 72, '1001': 178, '0001': 711, '1100': 256, '0100': 1474, '0110': 147, '1011': 29, '0010': 287}, {'1110': 25, '0000': 4393, '0111': 66, '1000': 459, '0101': 324, '0011': 325, '1010': 38, '1111': 20, '1101': 117, '1001': 208, '0001': 1420, '1100': 108, '0100': 284, '0110': 73, '1011': 21, '0010': 311}, {'1110': 20, '0000': 2431, '0111': 105, '1000': 315, '0101': 985, '0011': 367, '1010': 36, '1111': 30, '1101': 212, '1001': 354, '0001': 2532,

In [10]:
#Run in real quantum computer - Transpile user
#Misura di tutti i qubit
n = 4
q = QuantumRegister(n, 'q')
c = ClassicalRegister(n, 'c') #n

circ = QuantumCircuit(q,c)
phi = Parameter('phi')

circ.h(q[0])

for i in range(n-1):
    circ.cx(i, i+1)
    
circ.barrier()
circ.rz(phi, range(n))
circ.barrier()

for i in reversed(range(n-1)):
    circ.cx(i, i+1)

circ.h(q[0])

circ.barrier()

for i in range(n):
    circ.measure(q[i], c[i]) 

#circ.draw(output='mpl')

print('Phi range')
phi_range = [np.pi*j/(n+1) for j in range(2*n+1+1)]
print(phi_range)
circuits = [circ.bind_parameters({phi: phi_val}) for phi_val in phi_range] 
#print(circuits[3].parameters)
#circuits[11].draw(output='mpl')

layout = {q[0]: 13, q[1]: 12, q[2]: 2, q[3]:3}

backend = IBMQ.get_backend('ibmq_16_melbourne')
trans_ghz_mapped = transpile(circ, backend, initial_layout=layout)

job = execute(circ,
            backend,
            parameter_binds=[{phi: phi_val}
              for phi_val in phi_range], shots=shots)

job_monitor(job)

phi_range2 = phi_range
counts2 =[job.result().get_counts(i) for i in range(len(job.result().results))] 

print('\n')
print('Counts')
print(counts2)
print('\n')
print('Counts of 0000')
print(list(map(lambda c: 1/shots*c.get('0000', 0), counts2)))

job_id = job.job_id()
print('JOB ID: {}'.format(job_id))

#retrieved_job = backend.retrieve_job(job_id)
#retrieved_job.result().get_counts(qc)

Phi range
[0.0, 0.6283185307179586, 1.2566370614359172, 1.8849555921538759, 2.5132741228718345, 3.141592653589793, 3.7699111843077517, 4.39822971502571, 5.026548245743669, 5.654866776461628]
Job Status: job has successfully run


Counts
[{'1110': 18, '0000': 4197, '0111': 110, '1000': 402, '0101': 438, '0011': 297, '1010': 67, '1111': 20, '1101': 70, '1001': 288, '0001': 1267, '1100': 72, '0100': 251, '0110': 129, '1011': 50, '0010': 516}, {'1110': 22, '0000': 1106, '0111': 109, '1000': 250, '0101': 243, '0011': 338, '1010': 74, '1111': 23, '1101': 64, '1001': 410, '0001': 4613, '1100': 72, '0100': 306, '0110': 123, '1011': 57, '0010': 382}, {'1110': 21, '0000': 4931, '0111': 82, '1000': 461, '0101': 148, '0011': 383, '1010': 57, '1111': 15, '1101': 47, '1001': 212, '0001': 964, '1100': 63, '0100': 281, '0110': 108, '1011': 41, '0010': 378}, {'1110': 21, '0000': 2499, '0111': 101, '1000': 329, '0101': 155, '0011': 417, '1010': 69, '1111': 15, '1101': 59, '1001': 335, '0001': 3350, '110

In [9]:
n = 4
q = QuantumRegister(n, 'q')
c = ClassicalRegister(n, 'c') #n

circ = QuantumCircuit(q,c)
phi = Parameter('phi')

circ.h(q[0])

for i in range(n-1):
    circ.cx(i, i+1)
    
circ.barrier()
circ.rz(phi, range(n))
circ.barrier()

for i in reversed(range(n-1)):
    circ.cx(i, i+1)

circ.h(q[0])

circ.barrier()

for i in range(n):
    circ.measure(q[i], c[i]) 

#circ.draw(output='mpl')

print('Phi range')
phi_range = [np.pi*j/(n+1) for j in range(2*n+1+1)]
print(phi_range)
circuits = [circ.bind_parameters({phi: phi_val}) for phi_val in phi_range] 
#print(circuits[3].parameters)
#circuits[11].draw(output='mpl')

layout = {q[0]: 13, q[1]: 12, q[2]: 2, q[3]:3}

backend = IBMQ.get_backend('ibmq_16_melbourne')
trans_ghz_mapped = transpile(circ, backend, initial_layout=layout)

trans_ghz = transpile(circ, backend)
print(circ.depth())
print(trans_ghz.depth())
print(trans_ghz_mapped.depth())

Phi range
[0.0, 0.6283185307179586, 1.2566370614359172, 1.8849555921538759, 2.5132741228718345, 3.141592653589793, 3.7699111843077517, 4.39822971502571, 5.026548245743669, 5.654866776461628]
10
12
10
